<img src="https://i.ytimg.com/vi/BiPzz1xZ0XI/maxresdefault.jpg" width=800 height=600 />


Our datasets are usually a mix of categorical numerical and qualitative variables. In theory, numerical variables do not need any treatment to be passed to machine learning algorithms, but even so, some specific treatments, which we will discuss next, can bring some benefits during the training of our models. Qualitative (categorical) variables, on the other hand, require a pre-treatment in order to be transformed into numbers. We will see below some ways to do this, what are their strengths, weaknesses and we will discuss how to do this pre-processing in practice, with the help of ```scikit-learn``` pipelines.

Let's use the diabetes dataset for our examples.

# Setup

In [1]:
# libs
# category_encoder  --> !pip install category-encoders
# feature_engine use --> !pip install  feature-enginge
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder
from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

ERROR: Could not find a version that satisfies the requirement feature-enginge (from versions: none)
ERROR: No matching distribution found for feature-enginge


ModuleNotFoundError: No module named 'feature_engine'

# Data Preparation

The data preparation process predates the training of the machine-learning algorithm. The idea is to create a process that, for each input (feature/variable), performs a specific transformation. We will describe some of these transformations from now on, starting with numerical variables.

## Numerical Variables

The variables that are easiest to deal with are numerical variables, as machine-learning algorithms are already capable of interpreting them in their "raw" form. However, algorithms and their optimization processes can benefit from this transformation process. In general, machine learning algorithms do not understand that features are of different units and may assume that larger scale variables should be given greater "weight" as a result. Generally, algorithms that rely on calculating distances or gradient descents can suffer from the scaling factor. To avoid this kind of situation, we use scaling methods. Now we will present three transformation processes, their characteristics, strengths and weaknesses.

### MinMax Scaler

<img src="https://miro.medium.com/max/416/1*vEqbUwYneOkRQXCdPU0n9g.png" height=300 width=300/>


The first transformer that we will see is the MinMaxScaler, which is given by the expression below:

<img src=https://cdn-images-1.medium.com/max/800/0*K2QwZ16bEAxA4hUe.jpg height=300 width=300 />

Given a variable X, we will transform each value of X through the expression above.
for this we will need:
- Extract the Minimum from the X vector
- Extract the Max from the X vector

Note that the distribution format does not change!! Only the range of values ​​is changed, so that it behaves between 0 and 1.

> **Warning: Transformations need to be performed in training and testing!! However, we need to use the parameters obtained from the training base (minimum and maximum) to replicate the transformation in the test base. That is, when performing the transformation, we will use the minimum and maximum values ​​extracted from the training vector!!!**

But how is it in production? Do I need to store these values ​​somewhere?
Yes, of course.

But for that, ```scikit-learn``` has a series of Classes and Methods that will help us in this step. We will implement the ```MinMaxScaler``` class.


To use this transformer, we will need the ```.fit()``` and ```.transform()``` methods

In the ```.fit()``` method, the transformer extracts what it needs to perform the transformation. In our case, this is equivalent to extracting the maximum and minimum values ​​from the vector. For this, we will pass our vector as a parameter.

If we want to apply it to our entire training dataset, that is, to all our features at once, we can pass the entire training dataframe. Furthermore, the ```.fit_transform()``` method allows extracting the parameters and already applying the desired transformation.

In the end, we need to perform the transformation on our test set! To do this, just call the ```.transform()``` method


Disadvantage:
- The method is sensitive to outliers.

### Standard Scaler


We've already seen the Standard Scaler, when we talk about standardizing a normal curve.

<img src=https://cdn-images-1.medium.com/max/800/0*vQEjz0mvylP--30Q.GIF height=300 width=300/>


In this method, we extract the mean and standard deviation. From there, we subtract the mean of each sample and divide the result by the standard deviation.

After the transformation, the data are centered at 0 and have a standard deviation of 1.

Note that StandardScaler centers the data at 0. Also, there is no defined range, as in MinMaxScaler. However, the range of features is close.


If our data is normally distributed, algorithms can benefit more from using StandardScaler instead of MinMaxScaler.

Disadvantage:
- Sensitive to outliers

### Robust Scaler

A robust outlier method is the RobustScaler. This transformation is similar to the MinMax transformation, but uses Quartiles 1 and 3 to apply the scaling, instead of the maximum and minimum values.

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20200624151526/for4.png" height=300 width=300/>


The method implemented in ```scikit-learn``` subtracts the median (Q2) in place of Q1.


Note that the range changes, becomes better behaved, and that there are no outliers.

> **NOTE**: There is no golden rule or guarantee of improving the model using any transformer. We can only really say something after training the model. Furthermore, the same model can benefit from different transformers in different situations. However, it is worth noting that certain algorithms, such as PCA or KMeans, require the application of a specific transformer. Overall, the performances are similar to different transformers, but this is a setting that can be tweaked to result in the best possible performance.

## Qualitative Variables (Categorical)

Numerical variables are much easier to handle, they don't present many problems in production. When we talk about qualitative variables, then we do have potential problems for which we need to have some strategies.

The 2 main problems are:
- New levels in variables
- Rare levels

Before talking about these 2 problems, let's show how we can encode qualitative variables as numbers.


In this dataset, we have the following categorical variables:

- **Embarked**: Indicates the port where the passenger embarked
- **Sex**: Indicates the sex of the passenger
- **Pclass**: Class that the passenger traveled

### One-Hot Encoder

This first transformer we already know! That's right, we already know, but we call it ```pd.get_dummies()```


The idea is that for each level of the categorical variable a column is created, which can receive the value 0 or 1.

<img src="https://i.imgur.com/TW5m0aJ.png" height=700 width=700/>

Let's use ```OneHotEncoder``` to encode the variables


Disadvantages:
- If the variable has many levels (Brazil states), many columns will be created and this can be a problem in terms of processing and high dimensionality (curse of dimensionality).
- Some models (trees) benefit more from other methods.

### OrdinalEncoder


When there are ordinal categorical variables, these are best encoded by ```OrdinalEncoder```.
OrdinalEncoder assigns a number to each level of the variable, according to the order pre-established by the user.


Note that Poor < Good < Very Good < Excellent

If we don't specify the order of the levels, the transformer assigns the labels alphabetically in the levels.

### Frequency Encoder

When we have many levels in the categorical variables or even they do not have any ordinal degree, we can use the Frequency Encoder. This method replaces each class level with its frequency in the training data.


For example, in the variable Sex, male would be replaced by 0.65 and female by 0.35.

### Target Encoder

Finally, let's look at the Target Encoder. This method is similar to the previous one. However, instead of simply looking at the frequency of levels, it looks at the frequency of levels at the levels of the response variables, in the case of Target being categorical, and the average of Target, in case of Target being continuous.

# Column Transformer

So far we've seen how to apply transformations to categorical and numeric data separately. We would have to collect the results of our transformations and group them together to form a transformed set to insert into a Machine Learning model. Now we will learn how to perform all the processing in a single step, with the help of the ```Columns Transformer``` and ```Pipeline``` Classes


In it we have the variables:
- Embarked: Nominal Qualitative
- Sex: Nominal Qualitative
- Age: Continuous Numeric
- Pclass: Ordinal Qualitative
- Survived: Binary Qualitative Target


Therefore, we will use the following transformers:
- OrdinalEncoder: Pclass
- MinMaxScaler: Age
- OneHotEncoder: Sex, Embarked


Furthermore, at the end of the processing, we will use the ```Scikit-Learn``` Logistic Regression model


To build our universal transformer we need to create tuples.

The first element of the tuple will be an identifier, a string, which identifies the name of the process (it can be any name). The second element is the transformer itself and its properties, if any. The third element is a list with the name of the columns that will receive the processing.

Once that's done, we put everything in a list and pass it to the ```ColumnTransformer``` class


Finally, we will use the ```Pipeline``` class which will combine the preprocessing and training/prediction steps.

To this class we pass a list of tuples. The model always comes last!! Before it, any transformer can come, any class that has the fit and transform method.


Here we will pass the pre_processor we just created to our pipeline. This will be our step number 1, followed by step number 2, which will be the Logistic Regression model.

Let's split our data into training and testing.


With our pipeline built, we just need to call the ```.fit()``` method and then ```.predit()``` to make predictions with our model.


Shall we visualize our pre-processing and modeling pipeline?

> Note: In addition to handling data pre-processing, a modeling pipeline needs to handle filling in missing values ​​and handling new levels that may arise in categorical variables. For example, let's say we train a model to predict sales at points of sale that use PicPay. One of the variables used was the State to which the establishment belongs. However, PicPay expanded to new states and even states that did not participate in model training. Thus, even not having seen anything about that state, the model should be able to make some estimate. We will learn in classification classes how to deal with this and other problems related to categorical variables, such as the issue of rare levels.